In [1]:
import os
n = '5'
exportImage = 'Out\\'
exportData = 'Data\\'
motionImport = 'C:\\Users\\OndrejSpetko\\Desktop\\School\\MED7\\HRV-tracker\\Breathing\\motiondetection\\motiondetection\\Out\\'
sensorImport = 'C:\\Users\\OndrejSpetko\\Desktop\\School\\MED7\\HRV-tracker\\Breathing\\SerialListener\\Data\\'

In [2]:
#Motion file
from numpy import genfromtxt

motionRawData = genfromtxt(motionImport+n+'.txt', delimiter=',')
motionRawData.shape[0]

3846

In [3]:
#Sensor files
#sensorDatas = []
#for x in range(10):
#    temp = genfromtxt('C:\\Users\\OndrejSpetko\\Desktop\\School\\MED7\\HRV-tracker\\Breathing\\SerialListener\\Data\\'+str(x+1)+'.txt', delimiter=',')
#    sensorDatas.append(temp)

#for x in sensorDatas:
#    print(x.shape)

In [4]:
#Sensor file
FSR_IR = genfromtxt(sensorImport+n+'.txt', delimiter=',')
FSR_IR.shape[0]

1682

In [5]:
#Functions
def scale(a):
    scalar = (np.mean(a, axis=0) - np.mean(motionData, axis=0))/np.mean(motionData, axis=0)
    #print(scalar)
    out = np.array(a, dtype=float)/scalar*30
    return out

def move(a):
    out = a
    out -= min(out) 
    return out

In [6]:
#Match sizes
import numpy as np
import math

while motionRawData.shape[0] > FSR_IR.shape[0] :
    every = math.ceil(motionRawData.shape[0]/(motionRawData.shape[0] - FSR_IR.shape[0]))
    motionRawData = np.delete(motionRawData, list(range(0, motionRawData.shape[0], every)), axis=0)
motionRawData.shape

(1682, 2)

In [7]:
#Time data
timeData = motionRawData[:, 0]

In [8]:
# Motion data
from scipy import signal
temp = motionRawData[:, 1]
motionData = signal.savgol_filter(temp, 151, 3)
motionData = np.interp(motionData, (motionData.min(), motionData.max()), (0, 10))
#from sklearn import preprocessing
#motionDataNorm = np.array(motionData[:, 1])
#motionDataNorm = preprocessing.normalize([motionDataNorm])[0]
#motionDataNorm

In [9]:
# Normalize FSR data
#FSRData = scale(np.array(FSR_IR[:, 0]))
FSRData = np.array(FSR_IR[:, 0])
FSRData = signal.savgol_filter(FSRData, 151, 3)
FSRData = signal.savgol_filter(FSRData, 151, 3)
FSRData = np.interp(FSRData, (FSRData.min(), FSRData.max()), (0, 10))
#FSRData = move(FSRData)
#FSRDataNorm = preprocessing.normalize([temp])[0]
#FSRDataNorm.shape

In [10]:
# Normalize IR data
#IRData = scale(np.array(FSR_IR[:, 1]))
IRData = np.array(FSR_IR[:, 1])
IRData = signal.savgol_filter(IRData, 151, 3)
IRData = signal.savgol_filter(IRData, 151, 3)
IRData = np.interp(IRData, (IRData.min(), IRData.max()), (0, 10))
#IRData = move(IRData)
#IRDataNorm = preprocessing.normalize([temp])[0]
#IRDataNorm.shape

In [11]:
#Scatter plot
import plotly as py
import plotly.graph_objs as go
import plotly.io as pio

# Create traces
trace0 = go.Scatter(
    x = timeData,
    y = motionData,
    mode = 'lines',
    name = 'Motion'
)
trace1 = go.Scatter(
    x = timeData,
    y = FSRData,
    mode = 'lines',
    name = 'FSR'
)
trace2 = go.Scatter(
    x = timeData,
    y = IRData,
    mode = 'lines',
    name = 'IR'
)

#data = [trace0, trace1, trace2]
data = [trace0, trace1, trace2]

# Plot and embed in ipython notebook!
#pio.write_image(fig, exportImage+'fig1.png')
py.offline.plot(data, filename='Scatter.html')

'file://C:\\Users\\OndrejSpetko\\Desktop\\School\\MED7\\HRV-tracker\\Breathing\\PostProcessing\\Python\\Scatter.html'

In [12]:
#Correlation coefficient
#np.corrcoef(motionData, FSRData), np.corrcoef(motionData, IRData), np.corrcoef(FSRData, IRData)

In [13]:
#Bar plot correlation
corr1 = np.corrcoef(motionData, FSRData)[0][1]
corr2 = np.corrcoef(motionData, IRData)[0][1]
corr3 = np.corrcoef(FSRData, IRData)[0][1]
data2 = [go.Bar(
            x=['Motion vs. FSR', 'Motion vs. IR', 'IR vs. FSR'],
            y=[corr1, corr2, corr3],
            text=[corr1, corr2, corr3],
            textposition = 'auto',
)]

layout = go.Layout(
    title='Correlation Coefficient of Sensors',
)

fig = go.Figure(data=data2, layout=layout)
py.offline.plot(fig, filename='Bar.html')

'file://C:\\Users\\OndrejSpetko\\Desktop\\School\\MED7\\HRV-tracker\\Breathing\\PostProcessing\\Python\\Bar.html'

In [14]:
#Joining time + motion
temp = np.vstack((timeData, motionData))

size1 = temp[0].size
size2 = temp.size

temp = np.reshape(temp, size2, order='F')

temp = temp.reshape((size1, 2))
temp1 = temp
temp1.shape

(1682, 2)

In [15]:
#joining FSR + IR
temp = np.vstack((FSRData, IRData))
size1 = temp[0].size
size2 = temp.size

temp = np.reshape(temp, size2, order='F')

temp = temp.reshape((size1, 2))
temp2 = temp
temp2.shape

(1682, 2)

In [16]:
#Joining TimeMotion + FSRIR
final = np.hstack((temp1, temp2))
final, final.shape

(array([[ 0.28      ,  0.        ,  0.        , 10.        ],
        [ 0.36      ,  0.27988819,  0.22384225,  9.60265853],
        [ 0.44      ,  0.55430391,  0.44444647,  9.21793205],
        ...,
        [80.66551   ,  3.44317487,  3.30630382,  1.20020685],
        [80.744125  ,  3.39109523,  3.13588449,  1.15284312],
        [80.76      ,  3.33923308,  2.96325022,  1.10590941]]), (1682, 4))

In [17]:
#Saving to file
if not os.path.exists(exportData+n):
    os.mkdir(exportData+n)
np.savetxt(exportData+n+'\\'+n+'.txt', final, delimiter=",", fmt='%s')
np.savetxt(exportData+n+'\\'+n+'C.txt', [['MF', 'MI', 'FI'], [corr1, corr2, corr3]], delimiter=",", fmt='%s')
